In [ ]:
import sys, os 
import ipywidgets as widgets
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *
from IPython.display import display

In [ ]:
%%writefile healthcare_job_stocktf.sh

cd $PBS_O_WORKDIR 
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
source ~/anaconda3/etc/profile.d/conda.sh

conda activate stocktf
python3 healthcare.py -r $1
conda deactivate

In [ ]:
%%writefile healthcare_job_mkltf.sh

cd $PBS_O_WORKDIR 
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
source ~/anaconda3/etc/profile.d/conda.sh

conda activate mkltf
python3 healthcare.py -r $1 
conda deactivate

In [ ]:
core_stocktf = !qsub healthcare_job_stocktf.sh -l nodes=1:tank-870:i5-6500te  -F "results/"
print(core_stocktf[0]) 
if core_stocktf:
    progressIndicator('results/'+core_stocktf[0], 'i_progress.txt', "Processing", 0, 100)

In [ ]:
#Submit job to the queue
xeon_stocktf = !qsub healthcare_job_stocktf.sh -l nodes=1:tank-870:e3-1268l-v5 -F "results/"
print(xeon_stocktf[0]) 
if xeon_stocktf:
    progressIndicator('results/'+xeon_stocktf[0], 'i_progress.txt', "Processing", 0, 100)

In [ ]:
core_mkltf = !qsub healthcare_job_mkltf.sh -l nodes=1:tank-870:i5-6500te  -F "results/"
print(core_mkltf[0]) 
if core_mkltf:
    progressIndicator('results/'+core_mkltf[0], 'i_progress.txt', "Processing", 0, 100)

In [ ]:
#Submit job to the queue
xeon_mkltf = !qsub healthcare_job_mkltf.sh -l nodes=1:tank-870:e3-1268l-v5  -F "results/"
print(xeon_mkltf[0]) 
if xeon_mkltf:
    progressIndicator('results/'+xeon_mkltf[0], 'i_progress.txt', "Processing", 0, 100)

In [ ]:
outputHTML('IEI Tank (Intel Core CPU)',
          'results/'+core_stocktf[0], '.png')

In [ ]:
outputHTML('IEI Tank Xeon (Intel Xeon CPU)',
          'results/'+xeon_stocktf[0], '.png')

In [ ]:
outputHTML('IEI Tank (Intel Core CPU)',
          'results/'+core_mkltf[0], '.png')

In [ ]:
outputHTML('IEI Tank Xeon (Intel Xeon CPU)',
          'results/'+xeon_mkltf[0], '.png')

In [ ]:
arch_list = [('core_stocktf', 'Intel Core\ni5-6500TE\nCPU with MKL disabled'),
             ('xeon_stocktf', 'Intel Xeon\nE3-1268L v5\nCPU with MKL disabled'),
             ('core_mkltf', 'Intel Core\ni5-6500TE\nCPU with MKL enabled'),
             ('xeon_mkltf', 'Intel Xeon\nE3-1268L v5\nCPU with MKL enabled'),
             ]

stats_list = []
for job, a_name in arch_list:
    if job in vars():
        stats_list.append(('results/'+vars()[job][0]+'/stats.txt', a_name))
    else:
        stats_list.append(('placeholder'+job, a_name))

summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )

summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )